In [55]:
import os
import math

import pandas as pd
import numpy as np

## viz
from matplotlib import ticker
import matplotlib.pyplot as plt
import seaborn as sns

## toplines & visuals
import sweetviz as sv
# from pandas_profiling import ProfileReport

## set options for notebook
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## set options for viz
plt.style.use('ggplot')

# Data

In [56]:
surveys = pd.read_csv('./2016-survey-data-kaggle/VOTER_Survey_December16_Release1.csv')
surveys.columns = surveys.columns.str.lower()

/home/patruq/anaconda3/envs/dpp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (423,564) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
q1 = pd.read_csv('../job-apply/interviews/gsg/data/question1_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../job-apply/interviews/gsg/data/question1_data.csv'

In [ ]:
questions = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/questions.csv')
responses = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/responses.csv')
weight_schemes = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/weight_schemes.csv')
respondent_data = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/respondent_data.csv')
respondent_weight = pd.read_csv('../job-apply/interviews/gsg/data/question4_files/respondent_weights.csv')

In [ ]:
results = pd.read_csv('./election-results/results_sort_20141104.txt', sep='\t', header=None, low_memory=False)

# Toplines

In [ ]:
## QUESTION - RESPONSE - FREQUENCY - PERCENTAGE

In [ ]:
q_18058 = questions[questions['project_id'] == 18058].sort_values('question_id')
q_18390 = questions[questions['project_id'] == 18390].sort_values('question_id')

r_18058 = responses[responses['project_id'] == 18058].sort_values('question_id')
r_18390 = responses[responses['project_id'] == 18390].sort_values('question_id')

rd_18058 = respondent_data[respondent_data['project_id'] == 18058].sort_values('question_id')
rd_18390 = respondent_data[respondent_data['project_id'] == 18390].sort_values('question_id')

rw_18058 = respondent_weight[respondent_weight['project_id'] == 18058]
rw_18390 = respondent_weight[respondent_weight['project_id'] == 18390]

ws_18058 = weight_schemes[weight_schemes['project_id'] == 18058]
ws_18390 = weight_schemes[weight_schemes['project_id'] == 18390]

In [ ]:
## combine questions & response on 'responses'
output_18058 = r_18058.merge(q_18058, how='left', on='question_id')
## remove unneeded cols
output_18058 = output_18058[['question_id', 'response', 
                             'response_index', 'text_y','text_x']]
output_18058.rename(columns={
    'text_y': 'question_text',
    'text_x': 'response_text'
}, inplace=True)

## order the cols by question and respnse
output_18058.sort_values(['question_id', 'response'], inplace=True)

## corr resp_weights to weight scheme marked 'final' in weight_schemes
cond = (ws_18058['name'] == 'Final')
w_id = ws_18058[cond]['weight_scheme_id'].values[0]

rw_18058 = rw_18058[rw_18058['weight_scheme_id'] == w_id]

In [ ]:
## weights are tied to individual respondent id's
## For more info/background on weighted tables in survey data, ref'd from source:
    ## https://www.datarevelations.com/working-with-weighted-survey-data/
## perc is the % of overall choice in category/question

## combine the weights corr to respondent, response, and question.
m_weights_18058 = rd_18058.merge(rw_18058, how='left', 
                                 on='respondent_id').sort_values('respondent_id').groupby(['question_id', 
                                                                                           'response'], 
                                                                                           as_index=False).sum()
## merge with questions/respondents data
output_18058 = pd.merge(output_18058, m_weights_18058, how='left', 
                        left_on=['question_id', 'response'],
                        right_on=['question_id', 'response'])

## clean cols, round floats down
output_18058.rename(columns={'weight': 'freq_weight'}, inplace=True)
output_18058['freq_weight'].fillna(0, inplace=True)
output_18058['freq_weight'] = output_18058['freq_weight'].apply(lambda x: np.round(x, 1))

## the meat and potatoes:
cumsum = output_18058.groupby(['question_id']).sum().groupby(level=0).cumsum().reset_index()
output_18058 = output_18058.merge(cumsum, how='left', on='question_id')

output_18058['percent_weighted'] = output_18058['freq_weight_x'] / output_18058['freq_weight_y']

output_18058 = output_18058[['question_id', 'question_text', 'response_text', 
                             'freq_weight_x', 'percent_weighted']].rename(columns={'freq_weight_x': 'freq_weighted'})
output_18058

# Crosstabs, Pivot Tables

In [ ]:
print(surveys.shape)
surveys.head()

In [ ]:
cols = [col for col in surveys.columns if 'fav' in col and '_2016' in col] 
fav = surveys[cols].copy().drop(columns={'policies_favor_2016', 'fav_grid_row_rnd_2016', 'fav_grid_col_rnd_2016'})
fav.fillna('.', inplace=True)

## agg count the freq of values
fav = fav.apply(pd.value_counts)

fav.columns = ['Trump', 'Cruz', 'Ryan', 'Romney',
               'Obama', 'Clinton', 'Sanders', 'Rubio']

## swap the col and index of df
fav = fav.T

## convert to string with '%'
total = surveys.shape[0]
fav = fav.apply(lambda x: (np.round(x / total, 2)*100).astype(int).astype(str) + '%')

## reorder cols
fav = fav[['Very favorable', 'Somewhat favorable', 'Somewhat unfavorable', 'Very unfavorable', '.']]
fav

In [ ]:
## across all Q's
## age, gender, race, education, party ID, idealogy, 2016 vote choice, Dem vote choice, GOP vote choice
reg_col = 'votereg2_2016'
age_col = 'age'
gender_col = 'gender_baseline'
race_col = 'race_baseline'
edu_col = 'educ_baseline'
party_col = 'partyreg_baseline'
id_col = 'ideo5_2016'
vote_col = 'presvote16post_2016'
dem_col = 'pp_demprim16_2016'
gop_col = 'pp_repprim16_2016'

## create 'age' col 
tables = surveys.copy()

tables['age'] = (2016 - tables['birthyr_baseline'])

## CROSSTABS
    ## quantiles (buckets) for the ages
    ## NEED THIS FOR EITHER APPROACH
bins = [18, 30, 45, 64, np.inf]
labels = ['18-29', '30-44', '45-64', '65+']
tables['age_quant'] = pd.cut(tables['age'], bins=bins, labels=labels)

## LONG WAY
## group answers with age_quants
output = pd.DataFrame(tables.groupby('age_quant')[reg_col].value_counts()).unstack(level=[0])
## reset col names
output.columns = labels
## change col name
output.index.name = 'Voter Registration Status'
## sort index
output.sort_index(ascending=False, inplace=True)

## SHORT WAY
pd.crosstab(index=tables[reg_col], columns=tables['age_quant'])
    ## WITH %
pd.crosstab(index=tables[reg_col], columns=tables['age_quant']).apply(lambda x: (x/x.sum())*100, axis=1)

In [ ]:
    ## BUCKETS/QUANTS FOR CATEGORICAL DATA
tables.groupby(gender_col)[reg_col].value_counts().unstack(level=[0])
    ## or-
pd.crosstab(index=tables[gender_col], columns=tables[reg_col]).T

In [ ]:
results.columns = list(results.loc[0])
results.columns = results.columns.str.lower()
results = results.loc[1:]

cols = ['choice', 'contest name', 'choice party']
results[cols] = results[cols].apply(lambda x: x.str.lower())

results['total votes'] = results['total votes'].astype(int)
results['choice party'].fillna('', inplace=True)
results['id'] = results['choice'] + '_' + results['contest name'] + '_' + results['choice party']

In [ ]:
results = results.loc[(results['contest name'] == 'US SENATE'.lower()) | \
                     (results['contest name'].str.contains('NC SUPREME COURT'.lower())) | \
                     (results['contest name'].str.contains('NC COURT OF APPEALS'.lower()))]

In [ ]:
output = results.groupby(['county', 'id'], as_index=False)['total votes'].sum()

output = output.pivot(index='county', columns='id', values='total votes')

output

# Matches btwn cols

## map to new col based on value in other cols in 1 df

In [ ]:
## ref = use output from cleaning

output['age'] = output['age'].astype(int)

date_year = output['startdate'].str[-4].astype(int)
cond1 = (abs((date_year - output['d101']) - output['age']) == 1) | (abs((date_year - output['d101']) - output['age']) == 0)
cond2 = (output['sex'].map(lambda x: 2 if 'F' in x else 1) == output['d100'])

## new binary column of matches or mismatches
output['voter_file_match'] = (cond2 | (np.where(~cond1, False, True)))
output

# EDA Kickstarters

In [ ]:
report = sv.analyze(q1)
report.show_html()

# Cleaning

## split, strip values in col into new cols for each value

In [ ]:
## clean cols
q1.columns = q1.columns.str.lower()

## mask col
col = q1['vf_data']
## split the data in the col
split = col.apply(lambda x: x.split(';'))

## create dictionary of split values in previous col
dct = [dict(x.split('=', 1) for x in y) for y in split]

## merge dct into data
output = q1.join(pd.DataFrame(dct)).drop(columns='vf_data')
output.columns = output.columns.str.lower()

## order cols alphabetically

In [ ]:
col_order = ['origin']
new_order = col_order + (output.columns.drop(col_order).tolist())
output = output[new_order]

## BELOW
output = output.reindex(sorted(output.columns), axis=1)

# Vizuals

In [ ]:
## line plot

def name_plot(df, name):
    N = df.shape[0]
    data = df.loc[:, df.columns.str.contains(name)]
    
    plt.figure(figsize=(20,20))
    
    plt.plot(data.values, data.index)
    
    plt.grid()
    plt.xticks(fontsize=14, rotation=45)
    plt.title(name.upper())

name_plot(output, 'abe jones')

In [ ]:
## bar plot

## set race type
senate = output.loc[:, output.columns.str.contains('us senate')]
supreme_court = output.loc[:, output.columns.str.contains('nc supreme court')]
appeals_court= output.loc[:, output.columns.str.contains('nc court of appeals')]

# plt.figure(figsize=(80,80), dpi=80)

ax = senate.iloc[:10, :].plot(
    kind='barh', stacked=True, # stacked combines the results across candidates
    title='Results 2016'
)

fig = plt.gcf()
fig.set_size_inches(10.5, 12.5, forward=True)

ax.set(xlabel='Votes', ylabel='County')
ax.legend(title='Candidate')
plt.legend(loc='upper left', bbox_to_anchor=(1, 0.5))

In [ ]:
## change the output of pandas df

def color_nan_red(val):
    
    color = 'red' if math.isnan(val) else 'black'
    return 'color: %s' % color

senate.style.applymap(color_nan_red)

In [ ]:
## highlight cell in pandas df

def highlight_max(s):
    
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

senate.style.apply(highlight_max) # .apply operates col wise

In [ ]:
## call multiple functions over df
senate.style.\
    applymap(color_nan_red).\
    apply(highlight_max)

In [ ]:
## function to read either series or df
def highlight_max(data, color='yellow'):
    
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [58]:
senate.style.apply(highlight_max, color='darkorange', axis=None)

id,barry gurney (write-in)_us senate_,david waddell (write-in)_us senate_,john rhodes (write-in)_us senate_,kay hagan_us senate_dem,sean haugh_us senate_lib,thom tillis_us senate_rep,write-in (miscellaneous)_us senate_
county,,,,,,,
ALAMANCE,nan,nan,nan,18145.000000,1730.000000,24380.000000,73.000000
ALEXANDER,nan,1.000000,2.000000,3088.000000,833.000000,7925.000000,50.000000
ALLEGHANY,nan,nan,nan,1356.000000,329.000000,2496.000000,10.000000
ANSON,nan,nan,nan,4149.000000,232.000000,2374.000000,6.000000
ASHE,nan,nan,nan,3225.000000,594.000000,5658.000000,29.000000
AVERY,nan,nan,nan,1133.000000,225.000000,3718.000000,12.000000
BEAUFORT,nan,nan,nan,7071.000000,880.000000,9737.000000,86.000000
BERTIE,nan,nan,nan,4050.000000,163.000000,2218.000000,2.000000
BLADEN,nan,nan,nan,5639.000000,644.000000,5041.000000,23.000000
